In [1]:
import pandas as pd

In [2]:
books = pd.read_csv("books_with_categories.csv")

In [4]:
from transformers import pipeline
pipe = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None, device="mps")
pipe("I love this book")


Device set to use mps


[[{'label': 'joy', 'score': 0.9862679243087769},
  {'label': 'surprise', 'score': 0.005481816362589598},
  {'label': 'sadness', 'score': 0.003376907669007778},
  {'label': 'neutral', 'score': 0.002553066937252879},
  {'label': 'anger', 'score': 0.0012129780370742083},
  {'label': 'disgust', 'score': 0.0006926579517312348},
  {'label': 'fear', 'score': 0.0004146647988818586}]]

In [5]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [6]:
pipe(books["description"][0])

[[{'label': 'fear', 'score': 0.6548413634300232},
  {'label': 'neutral', 'score': 0.1698521375656128},
  {'label': 'sadness', 'score': 0.11640870571136475},
  {'label': 'surprise', 'score': 0.020700665190815926},
  {'label': 'disgust', 'score': 0.01910073310136795},
  {'label': 'joy', 'score': 0.015161280520260334},
  {'label': 'anger', 'score': 0.0039351508021354675}]]

In [7]:
pipe(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296023964881897},
  {'label': 'neutral', 'score': 0.14038589596748352},
  {'label': 'fear', 'score': 0.06816217303276062},
  {'label': 'joy', 'score': 0.04794251546263695},
  {'label': 'anger', 'score': 0.009156357496976852},
  {'label': 'disgust', 'score': 0.002628477755934},
  {'label': 'sadness', 'score': 0.0021221647039055824}],
 [{'label': 'neutral', 'score': 0.4493715167045593},
  {'label': 'disgust', 'score': 0.27359068393707275},
  {'label': 'joy', 'score': 0.10908294469118118},
  {'label': 'sadness', 'score': 0.09362741559743881},
  {'label': 'anger', 'score': 0.040478210896253586},
  {'label': 'surprise', 'score': 0.026970192790031433},
  {'label': 'fear', 'score': 0.006879056338220835}],
 [{'label': 'neutral', 'score': 0.6462160348892212},
  {'label': 'sadness', 'score': 0.2427334487438202},
  {'label': 'disgust', 'score': 0.0434226393699646},
  {'label': 'surprise', 'score': 0.02830049954354763},
  {'label': 'joy', 'score': 0.014211451634

In [8]:
senteces = books["description"][0].split(".")
predictions = pipe.predict(senteces)

In [9]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296023964881897},
 {'label': 'neutral', 'score': 0.14038589596748352},
 {'label': 'fear', 'score': 0.06816217303276062},
 {'label': 'joy', 'score': 0.04794251546263695},
 {'label': 'anger', 'score': 0.009156357496976852},
 {'label': 'disgust', 'score': 0.002628477755934},
 {'label': 'sadness', 'score': 0.0021221647039055824}]

In [11]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [14]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = pipe(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [15]:
emotion_scores

{'anger': [0.06413353234529495,
  0.6126179099082947,
  0.06413353234529495,
  0.35148343443870544,
  0.08141229301691055,
  0.2322252243757248,
  0.5381841659545898,
  0.06413353234529495,
  0.3006708025932312,
  0.06413353234529495],
 'disgust': [0.27359068393707275,
  0.34828516840934753,
  0.1040065586566925,
  0.15072256326675415,
  0.1844949573278427,
  0.7271744608879089,
  0.15585489571094513,
  0.1040065586566925,
  0.27948132157325745,
  0.17792607843875885],
 'fear': [0.9281686544418335,
  0.9425276517868042,
  0.9723208546638489,
  0.36070677638053894,
  0.09504333138465881,
  0.051362719386816025,
  0.747428297996521,
  0.40449604392051697,
  0.9155239462852478,
  0.051362719386816025],
 'joy': [0.9327977299690247,
  0.7044217586517334,
  0.7672373056411743,
  0.2518809139728546,
  0.04056433215737343,
  0.04337581247091293,
  0.8725650906562805,
  0.04056433215737343,
  0.04056433215737343,
  0.04056433215737343],
 'sadness': [0.6462160348892212,
  0.8879395127296448,
  0

In [16]:

from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = pipe(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5230/5230 [03:56<00:00, 22.12it/s]


In [17]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

books = pd.merge(books, emotions_df, on = "isbn13")

books = pd.merge(books, emotions_df, on = "isbn13")



In [18]:
books.to_csv("books_with_emotions.csv", index = False)